In [2]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [3]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [4]:
# default_exp fastai.data.load

In [8]:
# export
# Python native modules
import os
# Third party libs
from fastcore.all import *
from fastai.torch_basics import *
# from torch.utils.data.dataloader import DataLoader as OrgDataLoader
from torch.utils.data import dataloader
from torch.utils.data.dataloader import _MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter,_DatasetKind
_loaders = (_MultiProcessingDataLoaderIter,_SingleProcessDataLoaderIter)
# Local modules
from fastrl.fastai.loop import *

from fastai.data.load import DataLoader as FastaiDataLoader

from torchdata.datapipes.iter import HttpReader,IterDataPipe

# DataLoader helpers
> An implimentation of the torch DataLoader that instead uses Loop as a base. It
also uses experimental API from torch.data

In [6]:
bs = 4
letters = list(string.ascii_lowercase)

In [33]:
#export
def default_worker_f(worker_id):
    set_num_threads(1)
    info = get_worker_info()
    ds = info.dataset
    ds.offs = info.id
    set_seed(info.seed)
    if hasattr(ds,'wif'): ds.wif()

class MinimumDataLoader:
    def _fn_noops(self, x=None, *args, **kwargs): return x
    
    _IterableDataset_len_called,_auto_collation,collate_fn,drop_last = None,False,_fn_noops,False
    _index_sampler,generator,prefetch_factor = Inf.count,None,2
    dataset_kind = _dataset_kind = _DatasetKind.Iterable
    
    def __init__(self, 
                 datapipe, 
                 pin_memory=False, 
                 n_workers=0, 
                 timeout=0, 
                 persistent_workers=False
                ):
        self.dataset,self.worker_init_fn,self.num_workers = datapipe,default_worker_f,n_workers
        store_attr('pin_memory,timeout,persistent_workers')

    def __iter__(self): 
        
        return _loaders[self.num_workers==0](self)

    @property
    def multiprocessing_context(self): return (None,multiprocessing)[self.num_workers>0]

    @contextmanager
    def no_multiproc(self):
        old_num_workers = self.num_workers
        try:
            self.num_workers = 0
            yield self.d
        finally: self.num_workers = old_num_workers

_collate_types = (ndarray, Tensor, typing.Mapping, str)

In [34]:
for o in MinimumDataLoader(letters): print(o)

a
b
c
d
e
f
g
h
i
j
k
l
m
n
o
p
q
r
s
t
u
v
w
x
y
z


In [35]:

from torchdata.datapipes.iter import HttpReader,IterDataPipe,IterableWrapper

In [36]:
letter_pipe=IterableWrapper(letters).batch(2)
for o in MinimumDataLoader(letter_pipe): print(o)

['a', 'b']
['c', 'd']
['e', 'f']
['g', 'h']
['i', 'j']
['k', 'l']
['m', 'n']
['o', 'p']
['q', 'r']
['s', 't']
['u', 'v']
['w', 'x']
['y', 'z']


In [37]:
letter_pipe=IterableWrapper(letters).batch(2)
for o in MinimumDataLoader(letter_pipe,n_workers=3): print(o)

['a', 'b']
['a', 'b']
['a', 'b']
['c', 'd']
['c', 'd']
['c', 'd']
['e', 'f']
['e', 'f']
['e', 'f']
['g', 'h']
['g', 'h']
['g', 'h']
['i', 'j']
['i', 'j']
['i', 'j']
['k', 'l']
['k', 'l']
['k', 'l']
['m', 'n']
['m', 'n']
['m', 'n']
['o', 'p']
['o', 'p']
['o', 'p']
['q', 'r']
['q', 'r']
['q', 'r']
['s', 't']
['s', 't']
['s', 't']
['u', 'v']
['u', 'v']
['u', 'v']
['w', 'x']
['w', 'x']
['w', 'x']
['y', 'z']
['y', 'z']
['y', 'z']


In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()